## Test di Chow e Selezione dell'Ordine AR

Questo script serve ad indagare eventuali rotture strutturali nei dati.

1) Itera su un intervallo ragionevole di possibili ordini di lag per il modello AR della serie dell'inflazione.
2) Per ogni ordine, stima il modello AR.
3) Calcola l'AIC e il BIC per ciascun modello stimato.
4) Seleziona l'ordine di lag che minimizza uno di questi criteri (tipicamente si sceglie uno, ad esempio l'AIC, o si riportano entrambi e si discute la scelta).
5) Utilizza questo numero ottimale di lag per specificare il modello AR su cui verrà poi eseguito il test di Chow.

In [ ]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg, ar_select_order # Per la selezione dell'ordine AR
from scipy.stats import f # Per calcolare il p-value della statistica F

# --- CONFIGURAZIONE ---
# Path al file di input (output della Fase 2 - serie stazionarie)
PATH_INPUT_DIR_FASE2 = "/Users/tommaso/Desktop/tesi-inflation-gt/First_Difference_indexes/dati_preparati_fase2"
FILE_SERIE_STAZIONARIE_IN = os.path.join(PATH_INPUT_DIR_FASE2, "indici_gt_nic_stazionari_fase2.csv")

# Path per l'output dei risultati del Test di Chow
PATH_OUTPUT_CHOW_DIR = "/Users/tommaso/Desktop/tesi-inflation-gt/Analisi_Rottura_Chow"
FILE_CHOW_RISULTATI_OUT = os.path.join(PATH_OUTPUT_CHOW_DIR, "risultati_test_chow_2.csv")


# Nome della colonna dell'inflazione stazionaria (differenziata)
COL_INFLAZIONE_STAZ = 'NIC_destag_ISTAT_diff1'

# Punto di rottura ipotizzato (formato 'AAAA-MM-GG')
BREAKPOINT_GUERRA_UCRAINA = '2022-03-01'

# Numero massimo di lag da considerare per la selezione dell'ordine AR
MAX_AR_LAGS_SELEZIONE = 16 # Puoi aggiustare questo valore

SIGNIFICANCE_LEVEL_CHOW = 0.05

# --- FUNZIONI AUSILIARIE ---
def carica_dati_stazionari(path_file):
    """
    Carico le serie stazionarie (o trasformate) dal CSV salvato dalla Fase 2.
    """
    print(f"--- Caricamento Serie Stazionarie da: {path_file} ---")
    try:
        df = pd.read_csv(path_file, index_col=0)
        df.index = pd.to_datetime(df.index)
        print(f"Serie caricate con successo. Shape: {df.shape}")
        print("--- Fine Caricamento ---\n")
        return df
    except FileNotFoundError:
        print(f"ERRORE: File non trovato: {path_file}")
        return None
    except Exception as e:
        print(f"ERRORE durante il caricamento delle serie stazionarie: {e}")
        return None

def seleziona_ordine_ar_ottimale(y_series, max_lags, criterio='aic'):
    """
    Seleziona l'ordine ottimale per un modello AR usando AIC o BIC.
    y_series: la serie temporale (Pandas Series).
    max_lags: il numero massimo di lag da testare.
    criterio: 'aic' o 'bic'.
    """
    print(f"--- Selezione Ordine AR Ottimale (max_lags={max_lags}, criterio={criterio.upper()}) ---")
    y_cleaned = y_series.dropna()
    if len(y_cleaned) < max_lags + 10: # Controllo per dati sufficienti
        print("Dati insufficienti per una selezione affidabile dell'ordine AR. Uso un default di 4 lag.")
        return 4

    # Usiamo ar_select_order di statsmodels che è specifico per questo
    # 'noconstant' perché la costante verrà aggiunta dopo per il modello OLS del Chow test
    # 'trend=c' include una costante nel modello AR per la selezione dei lag
    selector = ar_select_order(y_cleaned, maxlag=max_lags, trend='c', ic=criterio, old_names=False)
    ordine_ottimale = selector.ar_lags # ar_lags restituisce una lista dei lag, prendiamo l'ultimo (il più alto) o la lunghezza
    
    if isinstance(ordine_ottimale, list) and ordine_ottimale:
        ordine_ottimale = ordine_ottimale[-1] # Se è una lista di lag, prendo il massimo
    elif isinstance(ordine_ottimale, int):
        pass # È già un intero
    else: # Fallback se l'output non è come atteso
        print(f"Attenzione: ar_select_order non ha restituito un ordine chiaro. Uso un default di 4 lag.")
        ordine_ottimale = 4
        
    print(f"Ordine AR ottimale selezionato tramite {criterio.upper()}: {ordine_ottimale}")
    print("--- Fine Selezione Ordine AR ---\n")
    return ordine_ottimale


def chow_test(y_series, X_series, breakpoint_date_str, num_params_k):
    """
    Esegue il test di Chow per un singolo punto di rottura.
    y_series: la variabile dipendente (Serie Pandas).
    X_series: le variabili indipendenti (DataFrame Pandas, inclusa intercetta).
    breakpoint_date_str: data del punto di rottura come stringa 'AAAA-MM-GG'.
    num_params_k: numero di parametri nel modello (inclusa l'intercetta).
    """
    print(f"--- Test di Chow per Rottura Strutturale al: {breakpoint_date_str} ---")
    
    data_full = pd.concat([y_series, X_series], axis=1).dropna()
    y_full = data_full.iloc[:, 0]
    X_full = data_full.iloc[:, 1:]
    
    n_full = len(y_full)
    if n_full < 2 * num_params_k + 2 :
        print("Dati insufficienti per eseguire il test di Chow.")
        return None, None, None, None, None

    breakpoint_ts = pd.Timestamp(breakpoint_date_str)
    if breakpoint_ts <= y_full.index.min() or breakpoint_ts >= y_full.index.max():
        print(f"Punto di rottura {breakpoint_date_str} fuori dal range dei dati. Test non eseguibile.")
        return None, None, None, None, None

    y1 = y_full[y_full.index < breakpoint_ts]
    X1 = X_full[X_full.index < breakpoint_ts]
    y2 = y_full[y_full.index >= breakpoint_ts]
    X2 = X_full[X_full.index >= breakpoint_ts]

    n1, n2 = len(y1), len(y2)
    if n1 < num_params_k + 1 or n2 < num_params_k + 1:
        print(f"Sottocampioni troppo piccoli (n1={n1}, n2={n2}, k={num_params_k}). Test non eseguibile.")
        return None, None, None, None, None
        
    try:
        model_R = sm.OLS(y_full, X_full).fit()
        rss_R = model_R.ssr
        
        model_1 = sm.OLS(y1, X1).fit()
        rss_1 = model_1.ssr
        
        model_2 = sm.OLS(y2, X2).fit()
        rss_2 = model_2.ssr
        
        numerator = (rss_R - (rss_1 + rss_2)) / num_params_k
        denominator = (rss_1 + rss_2) / (n1 + n2 - 2 * num_params_k)
        
        if denominator <= 1e-9: # Controllo per denominatore molto piccolo o zero
            print("Errore: Denominatore della statistica F nullo o troppo piccolo. Impossibile calcolare F.")
            return None, None, rss_R, rss_1, rss_2
            
        chow_f_statistic = numerator / denominator
        df_num = num_params_k
        df_den = n1 + n2 - 2 * num_params_k
        p_value = 1 - f.cdf(chow_f_statistic, df_num, df_den)
        
        print(f"RSS Modello Ristretto (intero campione): {rss_R:.4f}")
        print(f"RSS Sottocampione 1 (fino a {breakpoint_date_str}): {rss_1:.4f} (n1={n1})")
        print(f"RSS Sottocampione 2 (da {breakpoint_date_str} in poi): {rss_2:.4f} (n2={n2})")
        print(f"Statistica F di Chow: {chow_f_statistic:.4f}")
        print(f"Gradi di libertà: ({df_num}, {df_den})")
        print(f"P-value: {p_value:.4f}")
        
        if p_value <= SIGNIFICANCE_LEVEL_CHOW:
            print(f"RISULTATO: Si rifiuta H0. Evidenza di rottura strutturale al {SIGNIFICANCE_LEVEL_CHOW*100}% di significatività.")
        else:
            print(f"RISULTATO: Non si rifiuta H0. Nessuna evidenza significativa di rottura strutturale.")
        
        print("--- Fine Test di Chow ---\n")
        return chow_f_statistic, p_value, rss_R, rss_1, rss_2

    except Exception as e:
        print(f"ERRORE durante l'esecuzione del test di Chow: {e}")
        return None, None, None, None, None

# --- ESECUZIONE SCRIPT FASE 5 (Analisi Rotture Strutturali v2) ---
if __name__ == "__main__":
    print(">>> INIZIO SCRIPT FASE 5: Analisi Rotture Strutturali (Test di Chow v2) <<<\n")

    # Creo la directory di output per i risultati del Chow test se non esiste
    os.makedirs(PATH_OUTPUT_CHOW_DIR, exist_ok=True)
    print(f"Directory di output per i risultati del Chow test: '{PATH_OUTPUT_CHOW_DIR}'\n")

    # 1. Carico le serie stazionarie dalla Fase 2
    df_serie_stazionarie = carica_dati_stazionari(FILE_SERIE_STAZIONARIE_IN)

    if df_serie_stazionarie is None or COL_INFLAZIONE_STAZ not in df_serie_stazionarie.columns:
        print(f"ERRORE CRITICO: Impossibile caricare il file delle serie stazionarie o colonna '{COL_INFLAZIONE_STAZ}' mancante. Script interrotto.")
        exit()

    serie_inflazione = df_serie_stazionarie[COL_INFLAZIONE_STAZ].dropna() # Rimuovo eventuali NaN iniziali/finali

    if serie_inflazione.empty:
        print("ERRORE: Serie dell'inflazione vuota dopo dropna(). Script interrotto.")
        exit()

    # 2. Seleziono l'ordine AR ottimale per l'inflazione
    # Puoi scegliere 'aic' o 'bic' come criterio
    ar_lags_ottimali = seleziona_ordine_ar_ottimale(serie_inflazione, 
                                                  max_lags=MAX_AR_LAGS_SELEZIONE, 
                                                  criterio='aic') # o 'bic'
    
    if ar_lags_ottimali == 0: # ar_select_order potrebbe dare 0 se nessun lag è significativo
        print("Attenzione: Ordine AR ottimale selezionato è 0. Il test di Chow richiede almeno 1 lag per i regressori.")
        print("Imposto i lag a 1 per procedere, ma si consiglia di rivedere la serie o i criteri di selezione.")
        ar_lags_ottimali = 1


    # 3. Preparo i dati per il modello AR(p) dell'inflazione con l'ordine ottimale
    df_chow_data = pd.DataFrame()
    df_chow_data['y'] = serie_inflazione
    
    for i in range(1, ar_lags_ottimali + 1):
        df_chow_data[f'y_lag_{i}'] = df_chow_data['y'].shift(i)
    
    df_chow_data = sm.add_constant(df_chow_data, prepend=True, has_constant='skip') # Aggiungo costante se non già presente
    
    df_chow_data_cleaned = df_chow_data.dropna()
    
    y_chow = df_chow_data_cleaned['y']
    X_chow = df_chow_data_cleaned.drop(columns=['y'])
    
    num_parametri_modello = X_chow.shape[1]

    print(f"Modello per Test di Chow: {COL_INFLAZIONE_STAZ} ~ const + {ar_lags_ottimali} lag di se stessa.")
    print(f"Numero di parametri (k) nel modello: {num_parametri_modello}\n")

    if y_chow.empty or X_chow.empty or len(y_chow) < num_parametri_modello * 2 + 2: # Controllo aggiuntivo
        print("ERRORE: Dati insufficienti dopo la preparazione per il test di Chow con i lag selezionati. Script interrotto.")
        exit()
        
    risultati_chow_lista = [] # Lista per salvare i risultati

    # 4. Eseguo il Test di Chow per il punto di rottura (Guerra Ucraina - Marzo 2022)
    print(f"*** Test per rottura: Guerra in Ucraina - Shock Inflazionistico ({BREAKPOINT_GUERRA_UCRAINA}) ***")
    f_stat_guerra, p_val_guerra, rss_r_g, rss_1_g, rss_2_g = chow_test(y_chow, X_chow, BREAKPOINT_GUERRA_UCRAINA, num_parametri_modello)
    if f_stat_guerra is not None: # Salvo solo se il test è stato eseguito con successo
        risultati_chow_lista.append({
            'Breakpoint_Label': 'Guerra_Ucraina_Shock_Inflazionistico',
            'Breakpoint_Date': BREAKPOINT_GUERRA_UCRAINA,
            'F_Statistic': f_stat_guerra,
            'P_Value': p_val_guerra,
            'RSS_Ristretto': rss_r_g,
            'RSS_Sottocampione1': rss_1_g,
            'RSS_Sottocampione2': rss_2_g,
            'AR_Lags_Usati': ar_lags_ottimali
        })
        
    # 5. Salvo i risultati del test di Chow in un file CSV
    if risultati_chow_lista:
        df_risultati_chow = pd.DataFrame(risultati_chow_lista)
        try:
            df_risultati_chow.to_csv(FILE_CHOW_RISULTATI_OUT, index=False)
            print(f"\nRisultati del Test di Chow salvati con successo in: {FILE_CHOW_RISULTATI_OUT}")
            print("Contenuto del file dei risultati:")
            print(df_risultati_chow)
        except Exception as e_save:
            print(f"ERRORE durante il salvataggio dei risultati del Test di Chow: {e_save}")
    else:
        print("\nNessun risultato del Test di Chow da salvare (test potrebbe essere fallito o non eseguibile).")

    print("\n>>> SCRIPT FASE 5 COMPLETATO <<<")


>>> INIZIO SCRIPT FASE 5: Analisi Rotture Strutturali (Test di Chow v2) <<<

Directory di output per i risultati del Chow test: '/Users/tommaso/Desktop/tesi-inflation-gt/Analisi_Rottura_Chow'

--- Caricamento Serie Stazionarie da: /Users/tommaso/Desktop/tesi-inflation-gt/First_Difference_indexes/dati_preparati_fase2/indici_gt_nic_stazionari_fase2.csv ---
Serie caricate con successo. Shape: (252, 3)
--- Fine Caricamento ---

--- Selezione Ordine AR Ottimale (max_lags=16, criterio=AIC) ---
Ordine AR ottimale selezionato tramite AIC: 16
--- Fine Selezione Ordine AR ---

Modello per Test di Chow: NIC_destag_ISTAT_diff1 ~ const + 16 lag di se stessa.
Numero di parametri (k) nel modello: 17

*** Test per rottura: Guerra in Ucraina - Shock Inflazionistico (2022-03-01) ***
--- Test di Chow per Rottura Strutturale al: 2022-03-01 ---
RSS Modello Ristretto (intero campione): 24.9487
RSS Sottocampione 1 (fino a 2022-03-01): 9.0529 (n1=201)
RSS Sottocampione 2 (da 2022-03-01 in poi): 11.5255 (n2=34

/Users/tommaso/my_python_env/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/Users/tommaso/my_python_env/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
